In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# functions 
def get_upper(df):
    ser = df.sum(axis = 1)
    upper_number = sum(ser)*0.2
    sum_number = 0
    for i in ser[:: -1]:
        sum_number +=i
        if sum_number >= upper_number: 
            return ser[ser == i].index[0]
        
def get_middle(df):
    ser = df.sum(axis = 1)
    upper_number = sum(ser)*0.7
    sum_number = 0
    for i in ser[:: -1]:
        sum_number +=i
        if sum_number >= upper_number: 
            return ser[ser == i].index[0]
        
def merge_series(ser1, ser2, ser3): 
    """
    merge series into a dataframe 
    """
    dataframe = pd.DataFrame([ser1, ser2, ser3], 
                             columns = ser1.index, 
                             index = ['lower_income', 'middle_income', 'upper_income']) 
    return dataframe

def get_percent(data, cols, re_index): 
    """
    get the percentage of the rows sum
    """
    new_df = pd.DataFrame(columns = cols, index = re_index)
    for i in data.index:
        for j in data.columns:
            new_df.loc[i,j] = ((data.loc[i,j])/(data.sum(axis = 0)[j]))
    return new_df

def countplot(df, column, title, title_size): 
    plt.figure(figsize = (12,6))
    ncount = sum(df.drop(0)[column].value_counts())
    plt.xticks(rotation = 90)
    plt.title(title, fontsize = title_size)
    
    ax = sns.countplot(df.drop(0)[column])
    
    ax2 = ax.twinx()
    ax2.yaxis.tick_left()
    ax.yaxis.tick_right()
    
    ax.yaxis.set_label_position('right')
    ax2.yaxis.set_label_position('left')
    
    ax2.set_ylabel('Frequency [%]')
    
    for p in ax.patches:
        ax.annotate("{:.2%}".format(p.get_height()/ncount), (p.get_x()+0.4, p.get_height() + 100), 
               va = 'center', ha = 'center')
    return ax

## **1.Background and intention**

I am currently an international student studying in the American educational institution. Due to this pandemic and some financial problems, I took a gap leave from my college and am recently stuck at home. I do not want to waste my time waiting this Covid to end, so I took my first competition here at Kaggle to really build myself up with some data analysing skills. My intention to start with compensation is because I was wondering which factors can affect the compensation level one is earning or what things the income level can show. It turns out that it is a nice journey with data, and I really appreciate this opportunity that kaggle has offered. 



## **2. introduction**

As demonstrated in last part, this story will start with compensation level. among all of the data analysis graphs, I am particularly fond of heatmaps. It is probably because I am a math major and heatmap is just a form of matrix? Thus, I will also love to start off with a heatmap to demonstrate my point. In the first part, I will discover the relationship between coding experience and compensation. In part2, I will explore the relationship between computing langauges and compensation. In part3, I will show some of extra findings generated by part1 and part2. Finally, we will reach a conclusion to show what we have found through this project.  

## **3. Part1** 
This part we will explore the relation between coding experience and compensations. Regarding coding experience, it can be reflected by coding years. If you are coding longer, we assume that you are more experieced. Thus, I would love to create a heatmap following using compensation as y-axis and coding year as x- axis. 

In [ ]:
df = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')

# the data that I am using to produce the heatmap
cols = [
 'I have never written code',
 '< 1 years',
 '1-2 years',
 '3-5 years',
 '5-10 years',
 '10-20 years',
 '20+ years'
]

re_index =[
    '$0-999', '1,000-1,999', '2,000-2,999', '3,000-3,999', '4,000-4,999', '5,000-7,499', '7,500-9,999',
    '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999', '30,000-39,999', '40,000-49,999',
    '50,000-59,999', '60,000-69,999', '70,000-79,999', '80,000-89,999', '90,000-99,999', '100,000-124,999',
    '125,000-149,999', '150,000-199,999','200,000-249,999', '250,000-299,999','300,000-500,000', 
    '> $500,000']

df_drop = df.drop(0)

money_codeyear = df_drop.groupby(['Q24', 'Q6']).size().unstack(fill_value =0)
money_codeyear = money_codeyear[cols]
money_codeyear = money_codeyear.reindex(index = re_index)

In [ ]:
plt.figure(figsize = (20,10))
ax = sns.heatmap(money_codeyear, cmap = 'Blues', linewidth = 0.5, annot = True, fmt = 'd')
b, t = plt.ylim() 
b += 0.5 
t -= 0.5 
plt.ylim(b, t) 

ax1 = ax.hlines([3, 15], *ax.get_xlim())

textcolor = 'sienna'
ax.text(2.85,2, 'lower income', alpha = 0.6, size = 25, color = textcolor, family = 'sans-serif')
ax.text(2.8,8.5, 'middle income', alpha = 0.6, size = 25, color = textcolor, family = 'sans-serif')
ax.text(2.85,20.5, 'upper income', alpha = 0.6, size = 25, color = textcolor, family = 'sans-serif')

ax.annotate("",
            xy=(0.95,0.2), xycoords='axes fraction',
            xytext=(0.2, 0.95), textcoords='axes fraction',
            size=20, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple, head_length=2, head_width=3, tail_width=2",
                            color = 'darkseaGreen',
                            alpha = 0.5)
            )

ax.text(3.5,12.6, 'linear trend', rotation = -31, fontsize = 25, alpha = 1, color = 'white')
plt.show()

In [ ]:
print("The lower level of salary for kagglers attending these survet is from 0-999 to 3,000-3,999")

print ("The middle level of salary for kagglers attending these surveys is from", get_middle(money_codeyear), 
       'to', get_upper(money_codeyear))

print ("The top 20 percent of salary for kagglers attending these surveys is from", get_upper(money_codeyear), 
       "to", money_codeyear.sum(axis = 1).index[-1])


(1) we can clearly see there is almost a linear trend in this heatmap between coding years and compensation level. It seems that if you are coding for more years, you are more likely to be paid more. to analyze more closely, I will divide these people as three levels: upper income, middle income, and lower income by using the distribution of current American society class. which are approximately 20% of upper income, 50% of middle income, and 30% of lower income[1]. The following is the division. 

In [ ]:
lower_income = money_codeyear.iloc[0:3,:].sum(axis = 0)
middle_income = money_codeyear.iloc[3:15,:].sum(axis = 0)
upper_income = money_codeyear.iloc[15:,:].sum(axis = 0)
merge_lmu = merge_series(lower_income, middle_income, upper_income)

In [ ]:
plt.figure(figsize = (10, 6))
ax = sns.heatmap(merge_lmu, cmap = 'Blues', linewidth = 0.5, annot = True, fmt = 'd')
plt.setp(ax.get_yticklabels(), rotation = 0, rotation_mode = 'anchor')
b, t = plt.ylim() 
b += 0.5 
t -= 0.5 
plt.ylim(b, t) 

textcolor = 'sienna'
ax.text(4,0.5, 'pie chart 1', alpha = 1, size = 25, color = textcolor, family = 'sans-serif')
ax.text(4,1.5, 'pie chart 2', alpha = 1, size = 25, color = textcolor, family = 'sans-serif')
ax.text(4,2.5, 'pie chart 3', alpha = 1, size = 25, color = textcolor, family = 'sans-serif')


ax.annotate("",
            xy=(0.9,0.2), xycoords='axes fraction',
            xytext=(0.2, 0.8), textcoords='axes fraction',
            size=20, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple, head_length=2, head_width=2, tail_width=1",
                            color = 'darkseaGreen',
                            alpha = 0.5)
            )
ax.text(3,1.8, ' linear trend', rotation = -32.5, fontsize = 15, alpha = 1, color = 'white')
plt.show()

(2) To analyze more closely this relationship, I decide to merge this heatmap as above. In this way, we can see which group has contributed more to each level. we can see that from lower income to upper income, the weight in each level has shifted. if you are in the upper income, there is a higher possibility that you are coding for more than 5 years. if you are in the middle income, the main weight is in 2-10 years of coding. If you you are in a lower income, it is more likely that you are coding for 1-5 years. However, let us see this change more precisely in a pie chart to see how this has changed. 

In [ ]:
pie, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=[20,20])

labels = merge_lmu.columns
color = ['mistyrose', 'lightpink', 
        'pink', 'lightcoral',
        'indianred', 'red', 'darkred']

explodes = [0, 0, 0, 0, 0.1, 0.1, 0.1]

ax1.pie(x= merge_lmu.iloc[0,:],
        autopct="%.1f%%", 
        explode=explodes, 
        labels=labels, 
        colors = color, 
        pctdistance=0.5)

ax1.set_title('kagglers with lower income',fontsize = 14)

ax2.pie(x= merge_lmu.iloc[1,:], 
        autopct="%.1f%%", 
        explode=explodes, 
        labels=labels, 
        colors = color,
        pctdistance=0.5)

ax2.set_title('kagglers with middle income', fontsize=14)

ax3.pie(x= merge_lmu.iloc[2,:], 
        autopct="%.1f%%", 
        explode=explodes, 
        labels=labels,
        colors = color,
        pctdistance=0.5)

ax3.set_title('kagglers with higher income', fontsize=14)
plt.show()

(3) After transforming the heatmap to pie charts, the change becomes clearer visually. I emphasize three parts in these pie charts which are "5-10 years", "10-20 years", and "20+ years". As we can see in these pie charts, from lower to upper income, kagglers who have coded for  "5-10 years", "10-20 years", and "20+ years" are occupying the population increasingly. This fact says that among those coders, if you are in the higher class, there is a higher possibility that you are an experienced coder. Or in other words, if you have coded for more than 5 years, you are more likely to be paid more. 

## **4. Part2** 
In this part, what we are going to do is to explore the relation between compensations and the number of languages they use. It seemed to me that if someone knows more languages, it means that they(unknown gender) is more skillful so they will be paid more. During the process of analyzing, I will focus on only these three languages: **Python, R, and SQL.** Let us go deeper with some graphs. 

In [ ]:
cols =[
    '$0-999', '1,000-1,999', '2,000-2,999', '3,000-3,999', '4,000-4,999', '5,000-7,499', '7,500-9,999',
    '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999', '30,000-39,999', '40,000-49,999',
    '50,000-59,999', '60,000-69,999', '70,000-79,999', '80,000-89,999', '90,000-99,999', '100,000-124,999',
    '125,000-149,999', '150,000-199,999','200,000-249,999', '250,000-299,999','300,000-500,000', 
    '> $500,000'
]

re_idx = [
    'Python_R_SQL', 
    'Python_R',
    'Python_SQL',
    'Python',
    'R_SQL', 
    'R',
    'SQL', 
    'none'
]

order_idx =[
    'none', 
    'R', 
    'Python', 
    'SQL', 
    'Python_R',
    'Python_SQL', 
    'R_SQL', 
    'Python_R_SQL'
]


df_na = df.fillna('na') 
df_rsp = df_na.drop(0).groupby(['Q7_Part_1', 'Q7_Part_2', 'Q7_Part_3', 'Q24']).size().unstack()
df_rsp = df_rsp.drop('na', axis = 1).fillna(0)

dist_df = df_rsp.reset_index([0,1])
dist_df = dist_df.drop(['Q7_Part_1', 'Q7_Part_2'], axis = 1)
dist_df.index = re_idx
dist_df = dist_df.reindex(index = order_idx)
dist_df = dist_df[cols]

trans_df = dist_df.transpose()
no_language = trans_df.iloc[:,0]
one_language_only = trans_df.iloc[:,1:4].sum(axis = 1)
two_languages = trans_df.iloc[:,4:7].sum(axis = 1)
all_three = trans_df.iloc[:,7] 
df_merge = pd.DataFrame([no_language, one_language_only, two_languages, all_three], 
                       index = ['none', 'one_language', 'two_languages', 'three_languages'], 
                       )
df_merge = df_merge.transpose()

In [ ]:
plt.figure(figsize = (20,10))
ax = sns.heatmap(df_merge, cmap = 'Blues', linewidth = 0.5, annot = True, fmt = '.0f')
b, t = plt.ylim() 
b += 0.5 
t -= 0.5 
plt.ylim(b, t) 

ax1 = ax.hlines([3, 15], *ax.get_xlim())
ax.text(1.65,2, 'lower income', alpha = 0.6, size = 25, color = textcolor, family = 'sans-serif')
ax.text(1.65,8.5, 'middle income', alpha = 0.6, size = 25, color = textcolor, family = 'sans-serif')
ax.text(1.65,20.5, 'upper income', alpha = 0.6, size = 25, color = textcolor, family = 'sans-serif')

ax.annotate("",
            xy=(0.5,1.02), xycoords='axes fraction',
            xytext=(0,1.02), textcoords='axes fraction',
            size=20, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple, head_length=2, head_width=3, tail_width=2",
                            color = 'darkseaGreen',
                            alpha = 0.5)
            )

ax.text(0.3,-0.8, 'linear trend: Where to go? :(', fontsize = 15, alpha = 1, color = 'black')
plt.show()

(1) In this graph, we can not get any trend in this graph, the only information we can get in this graph is that for SQL, R, and Python,there are more kagglers who are using one or two of them compared to those who are using none or all of them as basis language. How can we go deeper in this situation if we would like to explore more about this relation? since the distribution is uneven, we can find the trend of maximum values for each column, so we can get the trend of how the weight is shifting among these columns. 

In [ ]:
plt.figure(figsize = (20,10))
ax = sns.heatmap(df_merge, cmap = 'Blues', linewidth = 0.5, annot = True, fmt = '.0f')
b, t = plt.ylim() 
b += 0.5 
t -= 0.5 
plt.ylim(b, t) 

patch_color = 'lightsalmon'
# first five maximum value in kagglers who know none of R, Python, and SQL
ax.add_patch(Rectangle((0,0),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((0,1),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((0,5),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((0,7),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((0,8),1,1, fill=False, edgecolor=patch_color, lw=3))

# first three maximum value in kagglers who know one of R, Python, and SQL
ax.add_patch(Rectangle((1,0),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((1,1),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((1,7),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((1,11),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((1,12),1,1, fill=False, edgecolor=patch_color, lw=3))

# first three maximum value in kagglers who know two of R, Python, and SQL
ax.add_patch(Rectangle((2,0),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((2,7),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((2,11),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((2,12),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((2,18),1,1, fill=False, edgecolor=patch_color, lw=3))

# first three maximum value in kagglers who know all of R, Python, and SQL
ax.add_patch(Rectangle((3,0),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((3,9),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((3,11),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((3,13),1,1, fill=False, edgecolor=patch_color, lw=3))
ax.add_patch(Rectangle((3,18),1,1, fill=False, edgecolor=patch_color, lw=3))

ax.annotate("",
            xy=(0.8,0.25), xycoords='figure fraction',
            xytext=(0.2,0.85), textcoords='figure fraction',
            size=20, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple, head_length=2, head_width=3, tail_width=2",
                            connectionstyle="arc3,rad=-0.2",
                            color = 'darkseaGreen',
                            alpha = 0.5)
            )

ax.text(1.7,10, 'maximum values trend :)', rotation = -27, fontsize = 15, alpha = 1, color = 'white') 

plt.show()

(2) Even though we cannot have a linear trend, we have got a non-linear trend in this graph. By following these colored squares in the heatmap, we know that from lower to upper income level, there is a higher possibility that they will be more familiar with R, SQL, and Python. in this graph, I picked five maximum values in each column, then, these five maximum values has shifted from up to the bottom. For example, if you do python, R, and SQL as regular bases, then your possibility to earn middle-income and upper-income salary will be higher than those who use only one of Python, R, and SQL as a regular basis. 

Then, we merge the heatmap again, notice that since we are choosing maximum values of each column as a trend, it makes more sense to transform the dataframe into percentage.

In [ ]:
lower_income1 = df_merge.iloc[0:3,:].sum(axis = 0)
middle_income1 = df_merge.iloc[3:15,:].sum(axis = 0)
upper_income1 = df_merge.iloc[15:,:].sum(axis = 0)
merge_lmu1 = merge_series(lower_income1, middle_income1, upper_income1)

cols = ['none', 
        'one_language', 
        'two_languages', 
        'three_languages']

re_index = ['lower_income',
            'middle_income', 
            'upper_income']

percent_lmu = get_percent(merge_lmu1, cols, re_index)

In [ ]:
plt.figure(figsize = (10, 6))
ax = sns.heatmap(percent_lmu.astype(float), cmap = 'Blues', linewidth = 0.5, annot = True, fmt = '.0%')
plt.setp(ax.get_yticklabels(), rotation = 0, rotation_mode = 'anchor')
b, t = plt.ylim() 
b += 0.5 
t -= 0.5 
plt.ylim(b, t)

ax.annotate("",
            xy=(0.9,0.2), xycoords='axes fraction',
            xytext=(0.2, 0.8), textcoords='axes fraction',
            size=20, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple, head_length=2, head_width=2, tail_width=1",
                            color = 'darkseaGreen',
                            alpha = 0.3)
            )
ax.text(1.7,1.8, ' linear trend', rotation = -32.5, fontsize = 15, alpha = 1, color = 'white')

textcolor = 'sienna'
ax.text(2,0.5, 'distplot 1', alpha = 1, size = 25, color = textcolor, family = 'sans-serif')
ax.text(2,2.5, 'distplot 2', alpha = 1, size = 25, color = textcolor, family = 'sans-serif')

plt.show()

(3) we can see a slight linear trend which moves the darker blues from upper left to lower right. However, notice that for all of the language-users, their highest value concentrates in middle_income, which means no matter how many languages they are doing as regular bases, it is more likely that they are earning middle-level income. However, if we only focus on lower_income and upper_income group, there is a interesting trend. 

In [ ]:
pie, ax = plt.subplots(1, 1, figsize=[10,6])

bar_width = 0.25
index = np.arange(4)
ax.bar(index, percent_lmu.iloc[0,:], bar_width, label = 'lower income')
ax.bar(index + bar_width, percent_lmu.iloc[2,:], bar_width, label = 'upper_income', color = 'darkseaGreen')
ax.legend()

ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(["none", "one_language", "two_languages", "three_languages"])
ax.set_ylabel('percentage',fontsize = 15)
ax.set_xlabel('number of languages', fontsize = 15)
ax.set_title('Comparison of lower income and upper income', fontsize= 20)

ax.annotate("",
            xy=(0.8,0.8), xycoords='axes fraction',
            xytext=(0.2, 0.5), textcoords='axes fraction',
            size=20, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple, head_length=1, head_width=1, tail_width=0.5",
                            color = 'darkseaGreen',
                            alpha = 0.6)
            )


ax.annotate("",
            xy=(0.8,0.7), xycoords='axes fraction',
            xytext=(0.15, 0.95), textcoords='axes fraction',
            size=20, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple, head_length=1, head_width=1, tail_width=0.5",
                            color = 'steelblue',
                            alpha = 0.6)
            )

ax.text(1.2,0.31, 'decreasing', rotation = -13, fontsize = 15, alpha = 1, color = 'black') 
ax.text(1.2,0.24, 'increasing', rotation = 14, fontsize = 15, alpha = 1, color = 'black') 

plt.show()

(4) Then, if we draw out the lower income and upper income alone, we can find out a different trends for them. If Kagglers are in the upper income level, there is a gradually increase trend saying that they will be more likely to use all these three languages as regular bases. And if they are in the lower income level, the trend is decreasing. Thus, numbers of computing languages may be a filter of higher and lower income in data science.   

## **5. Part3**
in this part, I will show useful extra visualizations that has been discovered during the process of this journey.

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize = (20,10))


sns.heatmap(df_merge, cmap = 'Blues', linewidth = 0.5, annot = True, fmt = '.0f', ax = ax1)
ax1.add_patch(Rectangle((0,0),4,1, fill=False, edgecolor=patch_color, lw=3)) 
ax1.text(1.5, -0.2, 'Student level', fontsize = 15, color = patch_color)

sns.heatmap(money_codeyear, cmap = 'Blues', linewidth = 0.5, annot = True, fmt = 'd', ax = ax2)
ax2.add_patch(Rectangle((0,0),7,1, fill=False, edgecolor=patch_color, lw=3))
ax2.text(3, -0.2, 'Student level', fontsize = 15, color = patch_color)


b, t = plt.ylim() 
b += 0.5 
t -= 0.5 
plt.ylim(b, t) 

plt.show()

(1) For kagglers, it is very unlikely that these these much of kagglers are being paid $0-999 anuanlly, Thus, I think the most proper reason for this is that they are still students. To verify this theory, let us see the distribution of professions.

In [ ]:
countplot(df, 'Q5', 'The Distribution of Professions', 15)
plt.show()

(2) This verifies our theory! Thus, if you are a student right now who is in need of money, probably graduating sooner is a not-bad move. 

In [ ]:
plt.figure(figsize = (20,10))
ax = sns.heatmap(dist_df.transpose(), cmap = 'Blues', linewidth = 0.5, annot = True, fmt = '.0f')
b, t = plt.ylim() 
b += 0.5 
t -= 0.5 
plt.ylim(b, t) 

ax.add_patch(Rectangle((2,0),1,25, fill=False, edgecolor='red', lw=3))
ax.add_patch(Rectangle((5,0),1,25, fill=False, edgecolor='red', lw=3))
ax.add_patch(Rectangle((7,0),1,25, fill=False, edgecolor='red', lw=3))

ax.annotate('Python', xy=(300, 550), xytext=(500, 600), xycoords='axes pixels', 
            ha='center', va='bottom', fontsize = 12,
            bbox=dict(boxstyle='square', fc='white'),
            arrowprops=dict(arrowstyle='-', lw=2.0))

ax.annotate('Python', xy=(600, 550), xytext=(500, 600), xycoords='axes pixels', 
            ha='center', va='bottom', fontsize = 12,
            bbox=dict(boxstyle='square', fc='white'),
            arrowprops=dict(arrowstyle='-', lw=2.0))

ax.annotate('Python', xy=(800, 550), xytext=(500, 600), xycoords='axes pixels', 
            ha='center', va='bottom', fontsize = 15,
            bbox=dict(boxstyle='square', fc='white'),
            arrowprops=dict(arrowstyle='-', lw=2.0))

ax.set_xlabel("Language they use on regular basis", fontsize = 15)
ax.set_ylabel("compensations", fontsize = 15)
plt.show()

(3) Another important information to be a data scientists is that to know Python. Through this heatmap, we can see that python plays a really important role in data analysis because you can see almost all columns with some blues are involved with Python. This means that for all kagglers who have attended this survey, most of them use python as a regular basis. No matter which social identity they are, it has been shown that Python now is required in most of areas. Therefore, the other key to be a successful data scientist is to be familiar with Python! 

## **Conclusion**

During the process of doing these data visualizations, I have improved my data analysizing skills, and I really appreciate this competition chance. 
In this project, we have explored this dataset with some graphs, and discovered some related aspects with the compensation level. To be clear, this project is not about cause and effect. This whole project is about trend and possibility. we only talk about which factors may increase your chance of being paid more, or being paid more may reflect some of the characteristics. 
Thus, the inducers of higher **possibilities** to have higher salary level are:
* Coding years 
* Numebrs of languages you use regularly(Python, R, SQL) 
* Not to be a student (Do not drop out)
* Be familiar with Python

## Reference
[1] Pew Research Center, "The American middle class is stable in size, but losing ground financially to upper-income families". SEP6, 2018. https://www.pewresearch.org/fact-tank/2018/09/06/the-american-middle-class-is-stable-in-size-but-losing-ground-financially-to-upper-income-families/